In [ ]:
import serial, time
ser = serial.Serial('/dev/ttyACM3', 115200)

In [ ]:
def read_all():
    r = ser.read_all() # clear buffer
    ser.write(b'a')
    while ser.in_waiting < 1:
        pass # wait for a response
    time.sleep(0.05)
    r = ser.read_all()
    t = str(r).split('A')[-1].strip()
    r = [[int(s.strip('b\'')) for s in str(l).split(',')[:16]] for l in str(r).split('A')[:-1]]
    return(r, t)

In [ ]:
import numpy as np
r, t = read_all()
np.asarray(r)

In [ ]:
base, t = read_all()

In [ ]:
r2, t = read_all()

In [ ]:
dif = np.asarray(base)-np.asarray(r2)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(dif)
plt.show()

In [ ]:
%%time
r2, t = read_all()
plt.plot((np.asarray(base)-np.asarray(r2))[0])
plt.ylim((-60, 60))
plt.show()

Running one with nothing

In [ ]:
r = []
y = []
for i in range(50):
    r2, t = read_all()
    r.append(np.asarray(base)-np.asarray(r2))
    y.append(0)

 And again with something in the ring

In [ ]:
for i in range(50):
    r2, t = read_all()
    r.append(np.asarray(base)-np.asarray(r2))
    y.append(1)

Building a modeland scoring it

In [ ]:
x = [rx.flatten() for rx in r]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(x, y)

regr = RandomForestClassifier(max_depth=5, random_state=0)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

In [ ]:
r2, t = read_all()
xt = []
xt.append((np.asarray(base)-np.asarray(r2)).flatten())
regr.predict(xt)

Not too bad, but not great either :/

# Image recon

In [ ]:
# Import required libraries
from image_util import *
import skimage.filters
from matplotlib import pyplot as plt
import cairocffi as cairo
import math, random
import numpy as np
import pandas as pd
from IPython.display import Image
from scipy.interpolate import interp1d
import astra

%matplotlib inline

def r8_to_sino(readings):
    readings = readings.tolist()
    sino = []
    for e in range(16):
        start = e*16 + (e+4)%16
        end = e*16 + (e+12)%16
        if end-start == 8:
            sino.append(readings[start : end])
        else:
            r = readings[start : (e+1)*16]
            for p in readings[e*16 : end]:
                r.append(p)
            sino.append(r)
    return np.asarray(sino)


In [ ]:
nviews = 16
ndetectors = 8
nvdetectors = 8

IMSIZE = 50
R = IMSIZE/2
D = IMSIZE/2

# Transforming from a round fan-beam to a fan-flat projection (See diagram)
beta = np.linspace(0, math.pi, ndetectors)
alpha = np.asarray([R*math.sin(b-math.pi/2)/(R**2 + D**2)**0.5 for b in beta])
tau = np.asarray([(R+D)*math.tan(a) for a in alpha])

tau_new = np.linspace(-(max(tau)/2), 1.3*max(tau)/2, nvdetectors)

vol_geom = astra.create_vol_geom(IMSIZE, IMSIZE)
angles = np.linspace(0,2*math.pi,nviews);
d_size = (tau[-1]-tau[0])/nvdetectors
proj_geom= astra.create_proj_geom('fanflat', d_size, nvdetectors, angles, D, R);
proj_id = astra.create_projector('line_fanflat', proj_geom, vol_geom)

In [ ]:
%%time
for i in range(1):
    print(i)
    r2, t = read_all()
    readings = (np.asarray(base)-np.asarray(r2)).flatten()# - base
    readings = r8_to_sino(readings) # Get important ones and reorder
    
    readings2 = []
    for r in readings:
        f = interp1d(tau, r, kind='cubic') # Can change to linear
        readings2.append(f(tau_new))
        
    
    sinogram_id = astra.data2d.create('-sino', proj_geom, np.asarray(readings2))
    
    # Plotting sinogram - new (transformed) set of readings
    plt.figure(num=None, figsize=(16, 10), dpi=80, facecolor='w', edgecolor='k')
    ax1 = plt.subplot(2, 3, 1)
    ax1.imshow(readings2) #<< Set title

    # Doing the reconstruction, in this case with FBP

    rec_id = astra.data2d.create('-vol', vol_geom)

    cfg = astra.astra_dict('FBP')
    cfg['ReconstructionDataId'] = rec_id
    cfg['ProjectionDataId'] = sinogram_id
    cfg['ProjectorId'] = proj_id

    # Create the algorithm object from the configuration structure
    alg_id = astra.algorithm.create(cfg)

    astra.algorithm.run(alg_id, 1)

    # Get the result
    rec = astra.data2d.get(rec_id)
    ax2 = plt.subplot(2, 3, 2)
    ax2.imshow(rec)
    norm_rec = rec/(np.amax(rec)*2)
    blurred = skimage.filters.gaussian(norm_rec, 3)
    ax3 = plt.subplot(2, 3, 3)
    ax3.imshow(blurred)
    
    plt.savefig('z' + str(i) + '.png')


# Trying ML for quadrant, with averaging of readings

In [ ]:
base = np.asarray(read_all()[0]).flatten()
for i in range(10):
    base = base+(np.asarray(read_all()[0]).flatten())
base = base/10

In [ ]:
read = np.asarray(read_all()[0]).flatten()
for i in range(10):
    read = read+(np.asarray(read_all()[0]).flatten())
read = read/10

plt.plot(base-read)
plt.ylim((-30, 30))

In [ ]:
def read_av():
    read = np.asarray(read_all()[0]).flatten()
    for i in range(10):
        read = read+(np.asarray(read_all()[0]).flatten())
    return read/10

read = read_av()
plt.plot(base-read)
plt.ylim((-30, 30))

In [299]:
r = []
y = []
for i in range(160):
    r.append(base-read_av())
    y.append(0)

In [306]:
for i in range(40):
    if i%10 == 0: print(i)
    r2 = read_av()
    r.append(base-read)
    y.append(4)

0
10
20
30


In [307]:
import pandas as pd

In [308]:
df = pd.DataFrame(r)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-5.2,7.3,-1.7,-7.5,-4.1,8.2,5.1,-9.9,17.9,-2.8,...,2.1,-11.9,-0.8,-16.3,-1.6,3.4,-10.0,-2.8,-5.7,-3.5
1,-7.0,-1.6,-10.3,2.9,-6.5,6.1,-2.2,-12.7,-3.2,4.5,...,1.3,-11.6,0.0,-6.0,7.1,16.8,-21.3,-3.1,5.3,-9.5
2,-5.5,-5.3,2.2,-6.5,-5.6,-3.3,-8.4,-15.6,0.1,-0.6,...,-8.4,1.1,-2.2,-10.2,-3.4,1.5,-16.6,-2.1,6.2,-5.7
3,-0.8,-0.3,5.4,-8.3,-3.1,7.8,0.0,-18.4,-6.2,8.1,...,1.2,-14.8,8.4,-2.7,-4.6,12.7,-11.1,6.4,-7.9,0.9
4,-7.5,-3.3,15.1,1.7,2.5,-7.8,5.2,-14.2,4.5,6.4,...,-3.9,-4.6,-0.1,-7.6,3.6,11.6,-9.4,5.8,-6.7,-3.6


In [309]:
df['Y'] = y
df.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,Y
0,-5.2,7.3,-1.7,-7.5,-4.1,8.2,5.1,-9.9,17.9,-2.8,...,-11.9,-0.8,-16.3,-1.6,3.4,-10.0,-2.8,-5.7,-3.5,0
1,-7.0,-1.6,-10.3,2.9,-6.5,6.1,-2.2,-12.7,-3.2,4.5,...,-11.6,0.0,-6.0,7.1,16.8,-21.3,-3.1,5.3,-9.5,0
2,-5.5,-5.3,2.2,-6.5,-5.6,-3.3,-8.4,-15.6,0.1,-0.6,...,1.1,-2.2,-10.2,-3.4,1.5,-16.6,-2.1,6.2,-5.7,0
3,-0.8,-0.3,5.4,-8.3,-3.1,7.8,0.0,-18.4,-6.2,8.1,...,-14.8,8.4,-2.7,-4.6,12.7,-11.1,6.4,-7.9,0.9,0
4,-7.5,-3.3,15.1,1.7,2.5,-7.8,5.2,-14.2,4.5,6.4,...,-4.6,-0.1,-7.6,3.6,11.6,-9.4,5.8,-6.7,-3.6,0


In [312]:
df.iloc[235:255]

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,Y
235,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,2
236,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,2
237,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,2
238,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,2
239,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,2
240,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,3
241,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,3
242,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,3
243,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,3
244,-3.7,-4.9,-1.1,6.9,-7.0,0.8,2.9,-20.2,-7.8,6.0,...,8.8,-0.7,-11.8,-4.3,5.2,-14.9,-0.1,5.8,-5.8,3


In [313]:
df.to_csv('r16_quadrant_test2.csv', index=False)

In [323]:
X_train, X_test, y_train, y_test = train_test_split(df[[i for i in range(256)]], df['Y'])

regr = RandomForestClassifier(max_depth=5, random_state=0)
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.625

In [336]:
x1 = r[160:240]
y1 = y[160:240]

y1 = [yx-1 for yx in y1]

X_train, X_test, y_train, y_test = train_test_split(x1, y1)

regr = RandomForestClassifier()
regr.fit(X_train, y_train)
regr.score(X_test, y_test)

0.5

In [333]:
y_test

[1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0]